In [18]:
'''
对抗训练
1. adversarial training
2. adversarial fine-tuning
'''
import pandas as pd
import numpy as np
import json
from datasets import Dataset
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
from sklearn.metrics import f1_score
import os
from torch.functional import F
from transformers import GPT2Tokenizer,GPT2ForSequenceClassification
from transformers import AdamW,get_linear_schedule_with_warmup

In [22]:
def adversari_training(model,tokenizer,train_data_path,valid_data_path,train_batch_size,eval_batch_size,num_epochs,lr,early_stopping,outputdir,adv_training_data_path,training_or_finetuning = "training"):
    '''
    当training_or_finetuning == "fine-tuning"时,进行adversarial fine-tuning
    否则,进行adversarial training
    '''
    traindf = pd.read_pickle(train_data_path)
    traindataset = Dataset.from_pandas(traindf)
    validdf = pd.read_pickle(valid_data_path)
    validdataset = Dataset.from_pandas(validdf)
    
    def tokenize_function(examples):
        return tokenizer(examples["text"],truncation = True,padding=True) 
    def collate_fn(examples):
        return tokenizer.pad(examples, padding="max_length", return_tensors="pt")
    train_tokenized_dataset = traindataset.map(tokenize_function,batched=True,remove_columns=["text","__index_level_0__"]).rename_column("label","labels")
    valid_tokenized_dataset = validdataset.map(tokenize_function,batched=True,remove_columns=["text","__index_level_0__"]).rename_column("label","labels")
    train_dataloader = DataLoader(train_tokenized_dataset,shuffle=True,collate_fn=collate_fn,batch_size = train_batch_size)
    eval_dataloader = DataLoader(valid_tokenized_dataset , collate_fn=collate_fn,batch_size = eval_batch_size)
    advdf = None
    advdataset = None
    if training_or_finetuning == "fine-tuning":
        advdf = pd.read_pickle(adv_training_data_path)
        advdataset = Dataset.from_pandas(advdf)
        adv_dataloader = DataLoader(advdataset , collate_fn=collate_fn,batch_size = train_batch_size)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model.to(device)
    model.resize_token_embeddings(len(tokenizer))
    trainlogdf = pd.DataFrame(columns=["step","trainloss","validloss","acc","f1-score"])
    rowindex = 0
    eval_no_progress_count = 0
    epochloss = []
    max_eval_acc = 0
    optimizer = AdamW(params=model.parameters(), lr=lr)
    iter_to_accumlate = 4
    # Instantiate scheduler

    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=0.06 * (len(train_dataloader) * num_epochs),
        num_training_steps=(len(train_dataloader) * num_epochs),
    )

    for epoch in range(num_epochs):
        model.train()
        allloss = 0
        for step,batch in enumerate(tqdm(train_dataloader)):
            batch.to(device)
            outputs = model(**batch)
            loss = outputs.loss/iter_to_accumlate
            loss.backward()
            allloss += loss.item()
            trainlogdf.loc[rowindex] = [rowindex,loss.item(),None,None,None]
            rowindex += 1
            epochloss.append(loss.item())
            if (step+1)%iter_to_accumlate==0:
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
            if (step+1)%(50*iter_to_accumlate) == 0:
                print("epoch",epoch,"step",step,"loss",loss,sep=" ")
            
            if (step+1)%800 == 0:
                count = 0
                model.eval()
                validloss = 0
                preds = []
                labels = []
                for evalstep,batch in enumerate(tqdm(eval_dataloader)):
                    labels += batch['labels'].cpu()
                    batch.to(device)
                    with torch.no_grad():
                        output = model(**batch)
                    validloss += output.loss.item()
                    pred = torch.argmax(F.softmax(output.logits.cpu(),dim=1),dim=1)
                    preds += pred
                    count += int(sum(batch['labels'].cpu() == pred))
                model.train()
                eval_acc = count/4000
                trainlogdf.loc[rowindex-1,"validloss"] = validloss/4000
                trainlogdf.loc[rowindex-1,"acc"] = eval_acc
                trainlogdf.loc[rowindex-1,"f1-score"] = f1_score(np.array(labels),np.array(preds),average="macro")
                print("epoch ",epoch,"step",step,"acc ",eval_acc)
                if eval_acc < max_eval_acc:
                    eval_no_progress_count += 1
                    if eval_no_progress_count >=early_stopping:
                        print("Early Stopping:Epoch",epoch," Step",step,"Eval_acc",eval_acc,sep=" ")
                        break
                    else:
                        print("Early Stopping record count",eval_no_progress_count,"Max eval acc",max_eval_acc,sep=" ")
                if eval_acc > max_eval_acc:
                    if training_or_finetuning == "fine-tuning":
                        '''
                        加载对抗样本进行训练
                        '''
                        model.train()
                        print("adver fine-tuning")
                        allloss = 0
                        for step,batch in tqdm(enumerate(adv_dataloader)):
                            batch.to(device)
                            output = model(**batch)
                            loss = output.loss
                            loss.backward()
                            allloss += loss.item()
                            optimizer.step()
                            lr_scheduler.step()
                            optimizer.zero_grad()
                        print("adver fine-tuning loss",allloss/(len(adv_dataloader)*train_batch_size))
                        
                    max_eval_acc = eval_acc
                    print("Update Max eval acc",max_eval_acc)
                    eval_no_progress_count = 0
                    model.save_pretrained(outputdir)
                    torch.save(model.state_dict(),os.path.join("checkpoint","model.bin"))
                    torch.save(optimizer.state_dict(),os.path.join("checkpoint","optimizer.bin"))
                    torch.save(lr_scheduler.state_dict(),os.path.join("checkpoint","lr_scheduler.bin"))
        print("epoch",epoch,"step",step,"trainLoss:",allloss/(len(train_dataloader)*train_batch_size))
    trainlogdf.to_csv("trainlog.csv")
    tokenizer.save_pretrained(outputdir)



In [23]:
model_path = "../../gpt2/"
model = GPT2ForSequenceClassification.from_pretrained(model_path,num_labels = 2)
tokenizer = GPT2Tokenizer.from_pretrained(model_path,bos_token = "<|startoftext|>",eos_token = "<|endoftext|>",pad_token = "<|pad|>",cls_token = "<|cls|>",sep_token = "<|sep|>" ,model_max_length = 1024)
adversari_training(model,tokenizer,"../dataset/train_adv.pkl","../dataset/valid.pkl",8,8,30,5e-5,5,"GPT2ADV_training_saved_models",None)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at ../../gpt2/ and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/ljc/anaconda3/envs/adv/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/11298 [00:01<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 model_path = "../../gpt2/"                                                                   │
│   2 model = GPT2ForSequenceClassification.from_pretrained(model_path,num_labels = 2)             │
│   3 tokenizer = GPT2Tokenizer.from_pretrained(model_path,bos_token = "<|startoftext|>",eos_t     │
│ ❱ 4 adversari_training(model,tokenizer,"../dataset/train_adv.pkl","../dataset/valid.pkl",8,8     │
│   5                                                                                              │
│                                                                                                  │
│ in adversari_training:48                                                                         │
│                                                                                                  │
│    45 │   │   allloss = 0                                                                        │
│    46 │   │   for step,batch in enumerate(tqdm(train_dataloader)):                               │
│    47 │   │   │   batch.to(device)                                                               │
│ ❱  48 │   │   │   outputs = model(**batch)                                                       │
│    49 │   │   │   loss = outputs.loss/iter_to_accumlate                                          │
│    50 │   │   │   loss.backward()                                                                │
│    51 │   │   │   allloss += loss.item()                                                         │
│                                                                                                  │
│ /home/ljc/anaconda3/envs/adv/lib/python3.8/site-packages/torch/nn/modules/module.py:1501 in      │
│ _call_impl                                                                                       │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /home/ljc/anaconda3/envs/adv/lib/python3.8/site-packages/transformers/models/gpt2/modeling_gpt2. │
│ py:1420 in forward                                                                               │
│                                                                                                  │
│   1417 │   │   """                                                                               │
│   1418 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return  │
│   1419 │   │                                                                                     │
│ ❱ 1420 │   │   transformer_outputs = self.transformer(                                           │
│   1421 │   │   │   input_ids,                                                                    │
│   1422 │   │   │   past_key_values=past_key_values,                                              │
│   1423 │   │   │   attention_mask=attention_mask,          

In [ ]:
adversari_training(model,tokenizer,"../dataset/train_adv.pkl","../dataset/valid.pkl",8,8,30,5e-5,5,"GPT2ADV_fine-tuning_saved_models","../dataset/train_adv.pkl","fine-tuning")